In [ ]:
!pip install spacy fhir.resources
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 2.8 MB/s eta 0:00:05
     ---- ----------------------------------- 1.6/12.8 MB 2.9 MB/s eta 0:00:04
     ------ --------------------------------- 2.1/12.8 MB 2.9 MB/s eta 0:00:04
     -------- ------------------------------- 2.6/12.8 MB 2.6 MB/s eta 0:00:04
     ---------- ----------------------------- 3.4/12.8 MB 2.8 MB/s eta 0:00:04
     ------------- -------------------------- 4.2/12.8 MB 3.0 MB/s eta 0:00:03
     -------------- ------------------------- 4.7/12.8 MB 2.9 MB/s eta 0:00:03
     ----------------- ---------------------- 5.5/12.8 MB 3.0 MB/s eta 0:00:03
     ------------------- -------------------- 6.3/12.8 MB 3.1 MB/s eta 0:00:03
     ---------------------- ----------------- 7.3/12.8 MB 3.2 MB/s eta 0:00:02
     ------------------------- -------------- 8.1/12.8 MB 3.3 MB/s

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
def extract_info(text):
    doc = nlp(text)
    age = None
    condition = None
    intent = None
    age_operator = '>'  # Default

    text_lower = text.lower()

    # Detect intent
    if any(keyword in text_lower for keyword in ["show", "list", "find"]):
        intent = "search"

    # Detect age comparison
    if any(x in text_lower for x in ["under", "below", "younger than"]):
        age_operator = '<'
    elif any(x in text_lower for x in ["over", "above", "older than"]):
        age_operator = '>'
    elif any(x in text_lower for x in ["of age", "age is", "age equal", "exact age", "age ="]):
        age_operator = '='

    # Extract age and condition
    for ent in doc.ents:
        if ent.label_ == "CARDINAL":
            try:
                age_candidate = int(ent.text)
                age = age_candidate
            except ValueError:
                pass
        if ent.label_ in ["DISEASE", "NORP", "CONDITION"] or any(x in ent.text.lower() for x in ["diabet", "cancer", "asthma"]):
            condition = ent.text.lower()

    # Fallback condition check
    if condition is None:
        for token in doc:
            if token.text.lower() in ["diabetes", "diabetic", "cancer", "asthma"]:
                condition = token.text.lower()

    if age is None:
        for token in doc:
            if token.text.isdigit():
                age = int(token.text)

    return {
        "intent": intent,
        "age": age,
        "age_operator": age_operator,
        "condition": condition
    }

In [ ]:
def simulate_fhir_request(parsed_info):
    age = parsed_info.get("age")
    age_operator = parsed_info.get("age_operator", ">")
    condition = parsed_info.get("condition")

    request = {
        "resourceType": "Bundle",
        "type": "searchset",
        "entry": []
    }

    if age is not None:
        request["entry"].append({
            "resource": {
                "resourceType": "Patient",
                "criteria": {
                    "age": f"{age_operator} {age}"
                }
            }
        })

    if condition:
        request["entry"].append({
            "resource": {
                "resourceType": "Condition",
                "criteria": {
                    "code": condition
                }
            }
        })

    return request

In [ ]:
test_queries = [
    "Show me all diabetic patients over 50",
    "List cancer patients above 40",
    "Find all patients with asthma older than 60",
    "Show all the patients under 45 who has cancer",
    "List all patients of age 30 with asthma"
]

for query in test_queries:
    print(f"\nQuery: {query}")
    parsed = extract_info(query)
    print("Parsed Info:", parsed)
    fhir_request = simulate_fhir_request(parsed)
    print("Simulated FHIR Request:")
    print(fhir_request)


Query: Show me all diabetic patients over 50
Parsed Info: {'intent': 'search', 'age': 50, 'age_operator': '>', 'condition': 'diabetic'}
Simulated FHIR Request:
{'resourceType': 'Bundle', 'type': 'searchset', 'entry': [{'resource': {'resourceType': 'Patient', 'criteria': {'age': '> 50'}}}, {'resource': {'resourceType': 'Condition', 'criteria': {'code': 'diabetic'}}}]}

Query: List cancer patients above 40
Parsed Info: {'intent': 'search', 'age': 40, 'age_operator': '>', 'condition': 'cancer'}
Simulated FHIR Request:
{'resourceType': 'Bundle', 'type': 'searchset', 'entry': [{'resource': {'resourceType': 'Patient', 'criteria': {'age': '> 40'}}}, {'resource': {'resourceType': 'Condition', 'criteria': {'code': 'cancer'}}}]}

Query: Find all patients with asthma older than 60
Parsed Info: {'intent': 'search', 'age': 60, 'age_operator': '>', 'condition': 'asthma'}
Simulated FHIR Request:
{'resourceType': 'Bundle', 'type': 'searchset', 'entry': [{'resource': {'resourceType': 'Patient', 'crite